In [1]:
import time

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

import config


In [2]:
driver = webdriver.Chrome()  # Mac用の記述
driver.implicitly_wait(10)


In [3]:
# SBIネオモバイルのログインページへアクセス
url_login = "https://trade.sbineomobile.co.jp/login"
driver.get(url_login)
time.sleep(3)  # ページに遷移する前に次の処理が実行されないようにするため


In [4]:
# ログインフォームの要素を取得する
username = driver.find_element(By.NAME, "username")
password = driver.find_element(By.NAME, "password")
login_btn = driver.find_element(By.ID, "neo-login-btn")

# 念のためテキストボックスの中身を空にする
username.clear()
password.clear()

# テキストボックスに値を入力する
username.send_keys(config.SBI_USERNAME)
password.send_keys(config.SBI_PASSWORD)

# ログインボタンをクリックする
login_btn.click()
time.sleep(1)


In [5]:
# SBIネオモバイルのポートフォリオ
url_portfolio = "https://trade.sbineomobile.co.jp/account/portfolio"
driver.get(url_portfolio)
time.sleep(3)


In [6]:
# 銘柄一覧の要素を取得する
div_element = driver.find_element(By.CLASS_NAME, "sp-main")

# 無限スクロールが終わるまでスクロールする
last_height = driver.execute_script("return arguments[0].scrollHeight", div_element)
while True:
    driver.execute_script(
        "arguments[0].scrollTo(0, arguments[0].scrollHeight);", div_element
    )
    time.sleep(2)
    new_height = driver.execute_script("return arguments[0].scrollHeight", div_element)
    if new_height == last_height:
        break
    last_height = new_height


In [7]:
# ページのhtmlを取得してパースする
html = driver.page_source.encode("utf-8")
parsed_html = BeautifulSoup(html, "html.parser")


In [8]:
# 保有銘柄の証券コード、銘柄名をそれぞれSeriesにする

# 証券コード
stock_code_list = []
tickers = parsed_html.find_all(class_="ticker")

for ticker in tickers:
    stock_code = ticker.get_text().strip()  # 抽出したテキストに空白がある場合は除去する
    stock_code_list.append(stock_code)

ser_stock_code = pd.Series(stock_code_list)

# 銘柄名
stock_name_list = []
names = parsed_html.find_all(class_="name")

for name in names:
    stock_name = name.get_text().strip()  # 抽出したテキストに空白がある場合は除去する
    stock_name_list.append(stock_name)

ser_stock_name = pd.Series(stock_name_list)


In [9]:
# 全銘柄の現在値〜預り区分をデータフレームのリストにする
table = parsed_html.find_all("table")
list_df_tables = pd.read_html(str(table))

# リストのデータフレームを一つに結合する
df_all_stock = pd.DataFrame()
for df_table in list_df_tables:
    row = df_table.T[1:2]
    df_all_stock = pd.concat([df_all_stock, row], axis=0)

# インデックスを振り直す
df_all_stock = df_all_stock.reset_index(drop=True)


In [10]:
# 証券コード、銘柄名をデータフレームに結合する
df_all_stock = pd.concat([ser_stock_code, ser_stock_name, df_all_stock], axis=1)

# カラム名を付け直す
df_all_stock.columns = ["コード", "名称", "現在値/前日比", "保有数量", "（うち売却注文中）", "評価損益率", "平均取得単価", "預り区分"]

In [17]:
# スプレッドシートのスキーマに合わせたデータフレームを作成する
columns = ["コード", "市場", "名称", "業種", "保有株式数", "購入単価", "現在単価"]
dtypes = {
    "コード": "object",
    "市場": "object",
    "名称": "object",
    "業種": "object",
    "保有株式数": "int64",
    "購入単価": "int64",
    "現在単価": "int64",
}
df_for_export = pd.DataFrame(columns=columns).astype(dtypes)


コード      object
市場       object
名称       object
業種       object
保有株式数     int64
購入単価      int64
現在単価      int64
dtype: object